System information (for reproducibility):

In [20]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [21]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2025spring/slides/19-easylineq`


Status `~/Documents/github.com/ucla-biostat-257/2025spring/slides/19-easylineq/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0
  [42fd0dbc] IterativeSolvers v0.9.4
  [7ed4a6bd] LinearSolve v3.9.0
  [b51810bb] MatrixDepot v1.0.13
  [af69fa37] Preconditioners v0.6.1
  [b8865327] UnicodePlots v3.7.2
  [efce3f68] WoodburyMatrices v1.0.0
  [37e2e46d] LinearAlgebra v1.11.0
  [9a3f8284] Random v1.11.0
  [2f01184e] SparseArrays v1.11.0


# Introduction

Consider $\mathbf{A} \mathbf{x} = \mathbf{b}$, $\mathbf{A} \in \mathbb{R}^{n \times n}$. Or, consider matrix inverse (if you want). $\mathbf{A}$ can be huge. Keep massive data in mind: 1000 Genome Project, NetFlix, Google PageRank, finance, spatial statistics, ... We should be alert to many easy linear systems. 

Don't blindly use `A \ b` and `inv` in Julia or `solve` function in R. **Don't waste computing resources by bad choices of algorithms!**

## Diagonal matrix

Diagonal $\mathbf{A}$: $n$ flops. Use `Diagonal` type of Julia.

In [22]:
using BenchmarkTools, LinearAlgebra, Random

# generate random data
Random.seed!(257)
n = 1000
A = diagm(0 => randn(n)) # a diagonal matrix stored as Matrix{Float64}
b = randn(n);

In [23]:
# should give link to the source code
@which A \ b

\(A::AbstractMatrix, B::AbstractVecOrMat)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.11.5+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/LinearAlgebra/src/generic.jl:1118

In [24]:
# check `istril(A)` and `istriu(A)` (O(n^2)), then call `Diagonal(A) \ b` (O(n))
@benchmark $A \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  361.958 μs …   7.896 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     367.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   385.810 μs ± 157.047 μs  ┊ GC (mean ± σ):  0.15% ± 1.62%

  ▇█▇▅▄▃▂▂▃▃▃▃▂▂▂▂▂▂▂▁▂▁▂▂▂▁▁ ▁ ▁                               ▂
  ████████████████████████████████▇▇█▇▆▆▆▆▇▇▆▆▅▆▆▅▅▅▆▅▃▅▄▄▅▅▄▄▄ █
  362 μs        Histogram: log(frequency) by time        507 μs <

 Memory estimate: 16.12 KiB, allocs estimate: 6.

In [25]:
# O(n) computation, no extra array allocation
@benchmark Diagonal($A) \ $b

BenchmarkTools.Trial: 10000 samples with 32 evaluations per sample.
 Range (min … max):  897.156 ns … 538.664 μs  ┊ GC (min … max):  0.00% … 99.47%
 Time  (median):       1.693 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.280 μs ±   8.607 μs  ┊ GC (mean ± σ):  26.26% ± 10.23%

   █                                                             
  ▅█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▂▂▂▂▂▂▂ ▂
  897 ns           Histogram: frequency by time         36.9 μs <

 Memory estimate: 16.12 KiB, allocs estimate: 6.

## Bidiagonal, tridiagonal, and banded matrices

Bidiagonal, tridiagonal, or banded $\mathbf{A}$: Band LU, band Cholesky, ... roughly $O(n)$ flops.   
* Use [`Bidiagonal`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.Bidiagonal), [`Tridiagonal`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.Tridiagonal), [`SymTridiagonal`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.SymTridiagonal) types of Julia.

In [26]:
Random.seed!(257) 

n  = 1000
dv = randn(n)
ev = randn(n - 1)
b  = randn(n) # rhs
# symmetric tridiagonal matrix
A  = SymTridiagonal(dv, ev)

1000×1000 SymTridiagonal{Float64, Vector{Float64}}:
 0.679063   0.817275    ⋅        …    ⋅          ⋅          ⋅ 
 0.817275   1.24568   -0.527485       ⋅          ⋅          ⋅ 
  ⋅        -0.527485  -1.21007        ⋅          ⋅          ⋅ 
  ⋅          ⋅         0.187263       ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
 ⋮                               ⋱                        
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅    

In [27]:
# convert to a full matrix
Afull = Matrix(A)

# LU decomposition (2/3) n^3 flops!
@benchmark $Afull \ $b

BenchmarkTools.Trial: 1003 samples with 1 evaluation per sample.
 Range (min … max):  4.409 ms …  12.590 ms  ┊ GC (min … max): 0.00% … 30.80%
 Time  (median):     4.631 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.984 ms ± 831.701 μs  ┊ GC (mean ± σ):  6.54% ± 10.07%

  ▆██▆                                                         
  ██████▆▆▅▃▃▂▂▂▁▃▄▃▃▃▃▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▃▂▃▃▂▂▂▂▂▂▁▂▂▂▁▁▂▂▂▂▁▂▂ ▃
  4.41 ms         Histogram: frequency by time        7.77 ms <

 Memory estimate: 7.67 MiB, allocs estimate: 9.

In [28]:
# specialized algorithm for tridiagonal matrix, O(n) flops
@benchmark $A \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  10.166 μs …  9.849 ms  ┊ GC (min … max): 0.00% … 99.69%
 Time  (median):     11.542 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.861 μs ± 98.381 μs  ┊ GC (mean ± σ):  7.63% ±  1.00%

  ▅▄▄▄▃▅██▆▅▆▇▇▆▅▄▃▂▁▁ ▁      ▁▁▁  ▁ ▁                        ▂
  ████████████████████████▇██▇█████████▇██▇▇█▆▆▆▄▅▅▄▄▅▃▂▃▄▅▄▅ █
  10.2 μs      Histogram: log(frequency) by time      18.9 μs <

 Memory estimate: 24.19 KiB, allocs estimate: 9.

## Triangular matrix

Triangular $\mathbf{A}$: $n^2$ flops to solve linear system.

In [29]:
Random.seed!(257)

n = 1000
A = tril(randn(n, n)) # a lower-triangular matrix stored as Matrix{Float64}
b = randn(n)

# check istril() then triangular solve
@benchmark $A \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  236.542 μs … 579.833 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     257.791 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   257.325 μs ±  22.295 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █       ▂█▁                                                    
  █▇▆▄▃▃▃▃███▇▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▂▂▁▁▁▂▁▁▂▂▁▂▂▂▂ ▃
  237 μs           Histogram: frequency by time          380 μs <

 Memory estimate: 8.06 KiB, allocs estimate: 3.

In [30]:
# triangular solve directly; save the cost of istril()
@benchmark LowerTriangular($A) \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  86.041 μs … 302.458 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     94.417 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   96.167 μs ±  12.210 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▂▂▁         █▆▇▅▃▂▄▃▃▃▂▁▂▁▁▂▁▁▂                             ▂
  ████▇▅█▇█▇▇▇▇████████████████████▇█▆▇▇▆▆▆▆▆▅▅▅▅▄▅▅▆▄▅▅▃▄▄▄▄▅ █
  86 μs         Histogram: log(frequency) by time       120 μs <

 Memory estimate: 8.06 KiB, allocs estimate: 3.

## Block diagonal matrix

Block diagonal: Suppose $n = \sum_b n_b$. For linear equations, $(\sum_b n_b)^3$ (without using block diagonal structure) vs $\sum_b n_b^3$ (using block diagonal structure).  

Julia has a [`blockdiag`](https://docs.julialang.org/en/v1/stdlib/SparseArrays/#SparseArrays.blockdiag) function that generates a **sparse** matrix. **Anyone interested writing a `BlockDiagonal.jl` package?**

In [31]:
using SparseArrays

Random.seed!(257)

B  = 10 # number of blocks
ni = 100
A  = blockdiag([sprandn(ni, ni, 0.01) for b in 1:B]...)

1000×1000 SparseMatrixCSC{Float64, Int64} with 998 stored entries:
⎡⡨⣶⣿⢼⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⢨⡿⣾⣞⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠉⠉⠁⢿⡺⣽⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⣻⣻⠯⣹⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠈⠀⠉⠁⣾⣼⣜⣏⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⣿⡷⣭⠞⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠀⣲⡩⢿⣞⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣯⣲⠽⢮⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠁⢟⣽⢟⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣿⡿⣭⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⣲⣾⣽⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣫⣿⣟⣿⠀⢀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣷⣺⣟⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⣵⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⣿⣻⣬⣟⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡧⣱⢾⣎⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠨⣫⣗⣱⡿⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⣾⣞⡿⡑⣀⠀⣀⢀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢘⣿⣳⣟⣉⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⡟⣿⢝⠟⎦

In [32]:
using UnicodePlots
spy(A)

         ┌──────────────────────────────────────────┐    
       1 │⡸⣞⣿⠽⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
         │⡹⣿⣼⣟⠅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
         │⠀⠁⠉⠁⢿⡻⣽⡷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⣽⢾⡗⢝⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠉⠉⠁⠉⢥⢶⣕⣒⠒⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⢺⡵⣜⡽⠗⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠘⠃⠠⡄⣲⣦⡖⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣻⣜⣫⢅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⠉⠙⠹⡴⣴⣰⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⡭⣿⣵⠯⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠙⠉⠛⣠⣀⣠⢄⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡿⠼⢿⣿⠅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠺⠛⠇⠺⢄⣀⢄⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣺⢦⣿⢾⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⠾⠾⠧⢃⣀⡀⣠⠄⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢽⢯⠷⡾⢥⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀

## Kronecker product

Use
$$
\begin{eqnarray*}
    (\mathbf{A} \otimes \mathbf{B})^{-1} &=& \mathbf{A}^{-1} \otimes \mathbf{B}^{-1} \\
    (\mathbf{C}^T \otimes \mathbf{A}) \text{vec}(\mathbf{B}) &=& \text{vec}(\mathbf{A} \mathbf{B} \mathbf{C}) \\
    \text{det}(\mathbf{A} \otimes \mathbf{B}) &=& [\text{det}(\mathbf{A})]^p [\text{det}(\mathbf{B})]^m, \quad \mathbf{A} \in \mathbb{R}^{m \times m}, \mathbf{B} \in \mathbb{R}^{p \times p}
\end{eqnarray*}    
$$
to avoid forming and doing costly computation on the potentially huge Kronecker $\mathbf{A} \otimes \mathbf{B}$.

**Anyone interested writing a package?**

In [33]:
using MatrixDepot, LinearAlgebra

A = matrixdepot("lehmer", 50) # a pd matrix

50×50 Matrix{Float64}:
 1.0        0.5        0.333333   0.25       …  0.0208333  0.0204082  0.02
 0.5        1.0        0.666667   0.5           0.0416667  0.0408163  0.04
 0.333333   0.666667   1.0        0.75          0.0625     0.0612245  0.06
 0.25       0.5        0.75       1.0           0.0833333  0.0816327  0.08
 0.2        0.4        0.6        0.8           0.104167   0.102041   0.1
 0.166667   0.333333   0.5        0.666667   …  0.125      0.122449   0.12
 0.142857   0.285714   0.428571   0.571429      0.145833   0.142857   0.14
 0.125      0.25       0.375      0.5           0.166667   0.163265   0.16
 0.111111   0.222222   0.333333   0.444444      0.1875     0.183673   0.18
 0.1        0.2        0.3        0.4           0.208333   0.204082   0.2
 0.0909091  0.181818   0.272727   0.363636   …  0.229167   0.22449    0.22
 0.0833333  0.166667   0.25       0.333333      0.25       0.244898   0.24
 0.0769231  0.153846   0.230769   0.307692      0.270833   0.265306   0.26
 ⋮  

In [34]:
B = matrixdepot("oscillate", 100) # pd matrix

100×100 Matrix{Float64}:
  0.707518      0.0578848    -0.00202508   …  -2.07618e-11   3.97326e-12
  0.0578848     0.433525      0.0956315        1.36333e-10  -2.60905e-11
 -0.00202508    0.0956315     0.807263        -1.45959e-10   2.79326e-11
  0.00464379   -0.0293052     0.0908958        8.28755e-10  -1.58602e-10
 -0.000796805   0.00596484   -0.00873108      -5.8795e-10    1.12518e-10
  0.000152015  -0.00249669    0.00519362   …   8.62146e-10  -1.64992e-10
  1.24633e-5    0.000187569  -0.00288517      -1.75966e-9    3.36752e-10
 -3.61382e-5   -3.52123e-5    0.000736403      1.38969e-9   -2.65949e-10
 -0.000252717   0.00129378   -0.000357227     -2.23571e-9    4.27855e-10
  2.10061e-5   -0.000141756   0.000188028      9.45806e-10  -1.81002e-10
  0.000199031  -0.000834268  -0.000535717  …  -1.90702e-9    3.64952e-10
  0.000217668  -0.00126173    0.000826701      2.78849e-9   -5.33643e-10
 -0.000162232   0.000941374  -0.000593161     -2.11532e-9    4.04816e-10
  ⋮                       

In [35]:
M = kron(A, B)
c = ones(size(M, 2)) # rhs
# Method 1: form Kronecker product and Cholesky solve
x1 = cholesky(Symmetric(M)) \ c;

In [36]:
# Method 2: use (A ⊗ B)^{-1} = A^{-1} ⊗ B^{-1}
m, p = size(A, 1), size(B, 1)
x2 = vec(transpose(cholesky(Symmetric(A)) \ 
    transpose(cholesky(Symmetric(B)) \ reshape(c, p, m))));

In [37]:
# relative error
norm(x1 - x2) / norm(x1)

9.089029324917874e-8

In [38]:
using BenchmarkTools

# Method 1: form Kronecker and Cholesky solve
@benchmark cholesky(Symmetric(kron($A, $B))) \ c

BenchmarkTools.Trial: 23 samples with 1 evaluation per sample.
 Range (min … max):  203.574 ms … 274.687 ms  ┊ GC (min … max): 0.22% … 2.29%
 Time  (median):     212.241 ms               ┊ GC (median):    2.99%
 Time  (mean ± σ):   218.385 ms ±  15.969 ms  ┊ GC (mean ± σ):  2.81% ± 0.60%

       █                                                         
  ▄▁▁▁▄█▆█▆▄▁▁▁▄▁▁▁▁▁▁▁▁▁▄▁▁▆▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  204 ms           Histogram: frequency by time          275 ms <

 Memory estimate: 381.56 MiB, allocs estimate: 10.

In [39]:
# Method 2: use (A ⊗ B)^{-1} = A^{-1} ⊗ B^{-1}
@benchmark vec(transpose(cholesky(Symmetric($A)) \ 
    transpose(cholesky(Symmetric($B)) \ reshape($c, p, m))))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  72.334 μs …  12.594 ms  ┊ GC (min … max):  0.00% … 99.22%
 Time  (median):     81.209 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   96.431 μs ± 249.243 μs  ┊ GC (mean ± σ):  14.60% ±  7.37%

  ▃▄▂▁▇█▆▅▄▂▁                                                  ▂
  █████████████▆▆▅▆▅▄▅▄▄▄▃▄▄▁▄▄▅▄▃▄▄▄▄▄▄▅▅▃▄▄▅▄▁▄▃▄▁▃▁▄▁▃▃▄▃▃▄ █
  72.3 μs       Histogram: log(frequency) by time       169 μs <

 Memory estimate: 244.50 KiB, allocs estimate: 18.

## Sparse matrix

Sparsity: sparse matrix decomposition or iterative method.  

* The easiest recognizable structure. Familiarize yourself with the sparse matrix computation tools in Julia, Matlab, R (`Matrix` package), MKL (sparse BLAS), ... as much as possible.

In [40]:
using MatrixDepot

Random.seed!(257)

# a 7701-by-7701 sparse pd matrix
A = matrixdepot("wathen", 50)
# random generated rhs
b = randn(size(A, 1))
Afull = Matrix(A)
count(!iszero, A) / length(A) # sparsity

0.001994776158751544

In [41]:
using UnicodePlots
spy(A)

         ┌──────────────────────────────────────────┐    
       1 │⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
         │⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
         │⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀

### Matrix-vector multiplication

In [42]:
# dense matrix-vector multiplication
@benchmark $Afull * $b

BenchmarkTools.Trial: 700 samples with 1 evaluation per sample.
 Range (min … max):  5.364 ms …  15.624 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.025 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.139 ms ± 760.020 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                       ▅▇█▅▂▁▃      ▂▆▅                        
  ▃▂▂▁▂▃▃▁▁▂▃▄▂▃▃▃▃▄▃▄▄███████▇▅▄▃▃▆█████▅▃▂▁▂▂▁▁▂▂▂▂▂▂▁▁▂▁▂▂ ▄
  5.36 ms         Histogram: frequency by time        9.02 ms <

 Memory estimate: 64.06 KiB, allocs estimate: 3.

In [43]:
# sparse matrix-vector multiplication
@benchmark $A * $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  61.875 μs …  16.609 ms  ┊ GC (min … max): 0.00% … 99.48%
 Time  (median):     66.875 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   72.315 μs ± 206.928 μs  ┊ GC (mean ± σ):  3.97% ±  1.41%

    ▃▂▁▃▇██▆▅▄▄▅▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁                               ▂
  ▅██████████████████████████████▇▇▇▇▇▇▆▇▇▆▆▆▆▅▆▅▅▅▄▂▃▄▃▄▄▃▃▂▅ █
  61.9 μs       Histogram: log(frequency) by time        96 μs <

 Memory estimate: 64.06 KiB, allocs estimate: 3.

### Solve linear equation

In [44]:
# solve via dense Cholesky
xchol = cholesky(Symmetric(Afull)) \ b
@benchmark cholesky($(Symmetric(Afull))) \ $b

BenchmarkTools.Trial: 8 samples with 1 evaluation per sample.
 Range (min … max):  624.373 ms … 721.384 ms  ┊ GC (min … max): 1.08% … 0.00%
 Time  (median):     643.595 ms               ┊ GC (median):    1.06%
 Time  (mean ± σ):   654.875 ms ±  32.720 ms  ┊ GC (mean ± σ):  0.91% ± 0.62%

  █  ██   █      █         █        █                         █  
  █▁▁██▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  624 ms           Histogram: frequency by time          721 ms <

 Memory estimate: 452.53 MiB, allocs estimate: 6.

In [45]:
# solve via sparse Cholesky
xcholsp = cholesky(Symmetric(A)) \ b
@show norm(xchol - xcholsp)
@benchmark cholesky($(Symmetric(A))) \ $b

norm(xchol - xcholsp) = 3.723067054378312e-15


BenchmarkTools.Trial: 685 samples with 1 evaluation per sample.
 Range (min … max):  5.756 ms … 32.488 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.676 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.296 ms ±  1.997 ms  ┊ GC (mean ± σ):  1.78% ± 2.89%

   ██▄▄▁                                                      
  ▄█████▇█▆▆▅▆▇▆▅▄▄▄▄▅▅▃▄▃▃▃▃▃▃▃▃▃▃▂▃▂▃▃▃▁▃▂▂▂▃▂▂▁▂▂▂▁▂▂▁▁▁▂ ▃
  5.76 ms        Histogram: frequency by time          13 ms <

 Memory estimate: 12.61 MiB, allocs estimate: 69.

In [46]:
# sparse solve via conjugate gradient
using IterativeSolvers

xcg = cg(A, b)
@show norm(xcg - xchol)
@benchmark cg($A, $b)

norm(xcg - xchol) = 2.5708139736888984e-7


BenchmarkTools.Trial: 313 samples with 1 evaluation per sample.
 Range (min … max):  15.684 ms …  19.290 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.961 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.991 ms ± 263.272 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁  ▄▅ ▄▅▄▃█▅▇▂▅                                          
  ▃▄▃▆▇█▇█████████████▆▆█▇▃▅▅▄▄▃▃▃▁▃▃▁▃▁▃▁▁▁▁▃▁▃▃▁▁▁▁▃▁▁▁▃▁▃▁▃ ▄
  15.7 ms         Histogram: frequency by time         16.8 ms <

 Memory estimate: 256.89 KiB, allocs estimate: 20.

In [47]:
# sparse solve via preconditioned conjugate gradient
using Preconditioners

xpcg = cg(A, b, Pl = DiagonalPreconditioner(A))
@show norm(xpcg - xchol)
@benchmark cg($A, $b, Pl = $(DiagonalPreconditioner(A)))

norm(xpcg - xchol) = 3.572940564591471e-8


BenchmarkTools.Trial: 1444 samples with 1 evaluation per sample.
 Range (min … max):  3.119 ms …  26.033 ms  ┊ GC (min … max): 0.00% … 87.38%
 Time  (median):     3.293 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.459 ms ± 865.474 μs  ┊ GC (mean ± σ):  0.46% ±  2.30%

  ▄█▇▇▆▅▅▃▄▂▁▁▁                                               ▁
  █████████████▇▇███▅▆▆▆▇▆▅▅▅▅▄▆▅▅▁▁▁▄▅▄▁▅▁▅▁▁▄▄▁▄▁▁▁▁▄▁▁▁▁▅▅ █
  3.12 ms      Histogram: log(frequency) by time      6.19 ms <

 Memory estimate: 256.89 KiB, allocs estimate: 20.

## Easy plus low rank

Easy plus low rank: $\mathbf{U} \in \mathbb{R}^{n \times r}$, $\mathbf{V} \in \mathbb{R}^{r \times n}$, $r \ll n$. Woodbury formula
\begin{eqnarray*}
	(\mathbf{A} + \mathbf{U} \mathbf{V}^T)^{-1} &=& \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_r + \mathbf{V} \mathbf{A}^{-1} \mathbf{U}^T)^{-1} \mathbf{V}^T \mathbf{A}^{-1} \\
    \text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) &=& \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_r + \mathbf{V} \mathbf{A}^{-1} \mathbf{U}^T).
\end{eqnarray*}

* Keep HW3 (multivariate density) and HW4 (PageRank) problems in mind.  

* [`WoodburyMatrices.jl`](https://github.com/timholy/WoodburyMatrices.jl) package can be useful.

In [48]:
using BenchmarkTools, Random, WoodburyMatrices

Random.seed!(257)
n = 1000
r = 5

A = Diagonal(rand(n))
B = randn(n, r)
D = Diagonal(rand(r))
b = randn(n)
# Woodbury structure: W = A + B * D * B'
W = SymWoodbury(A, B, D)
Wfull = Matrix(W) # stored as a Matrix{Float64}

1000×1000 Matrix{Float64}:
  1.59654    0.456106    1.52748   …  -0.725063   1.23432    -0.467818
  0.456106   3.23966     0.960678      1.22398   -0.256551   -0.191276
  1.52748    0.960678    3.39075       0.432445   1.34648    -1.06645
 -0.486356  -0.0292724  -1.36154      -0.998257  -0.378862    0.63341
 -1.12244    0.777027   -1.45956       1.42378   -1.84637     0.509516
  0.519072   1.93077     1.4578    …   1.88547   -0.250131   -0.539974
 -1.31967   -0.941968   -1.55701       1.24036   -1.73289     0.33383
 -0.365699   1.79832    -0.749824      1.03221   -0.893575    0.393119
 -0.206288  -1.30162    -0.764563     -1.75899    0.249943    0.334384
  0.598281   1.96579     2.04488       2.91997   -0.423147   -0.897774
 -1.26348   -1.97785    -1.99446   …  -0.589547  -1.19918     0.57366
 -0.220364  -1.3593     -1.02567      -1.63423    0.479068    0.404199
 -0.224507   0.206297   -0.473448     -0.372872  -0.595998    0.21142
  ⋮                                ⋱                   

In [49]:
# compares storage
Base.summarysize(W), Base.summarysize(Wfull)

(48480, 8000048)

### Solve linear equation

In [50]:
# solve via Cholesky
@benchmark cholesky($(Symmetric(Wfull))) \ $b

BenchmarkTools.Trial: 1210 samples with 1 evaluation per sample.
 Range (min … max):  3.017 ms … 25.034 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     3.541 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.130 ms ±  1.480 ms  ┊ GC (mean ± σ):  7.01% ± 12.30%

     █▇▅▄▃▂▁▁▁▁   ▁▂▂                                         
  ▄▄▇██████████▇█▇█████▇██▇▇▇▇▇▇▇▆▆▆▇▆▆▆▁▆▅▄▅▆▅▅▄▁▅▄▁▄▄▄▄▅▁▄ █
  3.02 ms      Histogram: log(frequency) by time     9.24 ms <

 Memory estimate: 7.66 MiB, allocs estimate: 6.

In [51]:
# solve using Woodbury formula
@benchmark $W \ reshape($b, n, 1) # hack; need to file an issue 

BenchmarkTools.Trial: 10000 samples with 6 evaluations per sample.
 Range (min … max):  5.521 μs …  3.830 ms  ┊ GC (min … max):  0.00% … 99.71%
 Time  (median):     7.653 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   9.279 μs ± 58.956 μs  ┊ GC (mean ± σ):  16.63% ±  3.92%

                      ██▆▇▅▅▃▃▁                               
  ▂▂▃▃▃▂▂▁▁▁▂▂▂▂▂▂▃▃▅▇██████████▇▆▅▄▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁ ▃
  5.52 μs        Histogram: frequency by time        10.5 μs <

 Memory estimate: 32.58 KiB, allocs estimate: 17.

### Matrix-vector multiplication

In [52]:
# multiplication without using Woodbury structure
@benchmark $Wfull * $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):   73.208 μs …  2.262 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     123.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   143.964 μs ± 71.516 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▄▃  █▇                                                     
  ▁▁▂██▄▄█████▃▃▅▅▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  73.2 μs         Histogram: frequency by time          412 μs <

 Memory estimate: 8.06 KiB, allocs estimate: 3.

In [53]:
# multiplication using Woodbury structure
@benchmark $W * $b

BenchmarkTools.Trial: 10000 samples with 9 evaluations per sample.
 Range (min … max):  2.542 μs …  3.039 ms  ┊ GC (min … max):  0.00% … 99.77%
 Time  (median):     4.685 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.174 μs ± 46.070 μs  ┊ GC (mean ± σ):  23.48% ±  5.43%

                         ▁▅█▇█▇▅▄▃▂▁                          
  ▂▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▃▅▅▆▇████████████▇▆▆▆▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁ ▃
  2.54 μs        Histogram: frequency by time        6.88 μs <

 Memory estimate: 32.44 KiB, allocs estimate: 16.

### Determinant

In [54]:
# determinant without using Woodbury structure
@benchmark det($Wfull)

BenchmarkTools.Trial: 759 samples with 1 evaluation per sample.
 Range (min … max):  3.540 ms … 78.536 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     4.159 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.590 ms ±  7.218 ms  ┊ GC (mean ± σ):  5.37% ± 11.87%

  █▁▂▁▃▄▄▁▁                                                   
  ██████████▇▆▆▅▅▄▁▄▁▄▄▄▁▁▁▁▁▅▅▁▁▁▁▁▁▁▁▁▁▁▁▄▁▅▁▁▁▁▁▁▁▁▁▁▁▄▁▄ ▇
  3.54 ms      Histogram: log(frequency) by time     43.9 ms <

 Memory estimate: 7.66 MiB, allocs estimate: 6.

In [55]:
# determinant using Woodbury structure
@benchmark det($W)

BenchmarkTools.Trial: 10000 samples with 232 evaluations per sample.
 Range (min … max):  310.884 ns … 114.323 μs  ┊ GC (min … max): 0.00% … 99.59%
 Time  (median):     356.681 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   384.452 ns ±   1.295 μs  ┊ GC (mean ± σ):  5.19% ±  1.72%

                 ▂██▆▃                                           
  ▁▁▁▂▁▁▁▁▂▃▃▃▂▂▅██████▇▇▆▅▄▃▃▃▃▃▂▂▂▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  311 ns           Histogram: frequency by time          457 ns <

 Memory estimate: 368 bytes, allocs estimate: 4.

## Easy plus border

Easy plus border: For $\mathbf{A}$ pd and $\mathbf{V}$ full row rank,
$$
	\begin{pmatrix}
	\mathbf{A} & \mathbf{V}^T \\
	\mathbf{V} & \mathbf{0}
	\end{pmatrix}^{-1} = \begin{pmatrix}
	\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{V}^T (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \mathbf{V} \mathbf{A}^{-1} & \mathbf{A}^{-1} \mathbf{V}^T (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \\
	(\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \mathbf{V} \mathbf{A}^{-1} & - (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1}
	\end{pmatrix}.
$$
**Anyone interested writing a package?**

## Orthogonal matrix

Orthogonal $\mathbf{A}$: $n^2$ flops **at most**. Why? Permutation matrix, Householder matrix, Jacobi matrix, ... take less.

## Toeplitz matrix

Toeplitz systems (constant diagonals):
$$
	\mathbf{T} = \begin{pmatrix}
	r_0 & r_1 & r_2 & r_3 \\
	r_{-1} & r_0 & r_1 & r_2 \\
	r_{-2} & r_{-1} & r_0 & r_1 \\
	r_{-3} & r_{-2} & r_{-1} & r_0
	\end{pmatrix}.
$$
$\mathbf{T} \mathbf{x} = \mathbf{b}$, where $\mathbf{T}$ is pd and Toeplitz, can be solved in $O(n^2)$ flops. Durbin algorithm (Yule-Walker equation), Levinson algorithm (general $\mathbf{b}$), Trench algorithm (inverse). These matrices occur in auto-regressive models and econometrics.

* [`ToeplitzMatrices.jl`](https://github.com/JuliaMatrices/ToeplitzMatrices.jl) package can be useful.

## Circulant matrix

Circulant systems: Toeplitz matrix with wraparound
$$
	C(\mathbf{z}) = \begin{pmatrix}
	z_0 & z_4 & z_3 & z_2 & z_1 \\
	z_1 & z_0 & z_4 & z_3 & z_2 \\
	z_2 & z_1 & z_0 & z_4 & z_3 \\
	z_3 & z_2 & z_1 & z_0 & z_4 \\
	z_4 & z_3 & z_2 & z_1 & z_0
	\end{pmatrix},
$$
FFT type algorithms: DCT (discrete cosine transform) and DST (discrete sine transform).

## Vandermonde matrix

Vandermonde matrix: such as in interpolation and approximation problems
$$
	\mathbf{V}(x_0,\ldots,x_n) = \begin{pmatrix}
	1 & 1 & \cdots & 1 \\
	x_0 & x_1 & \cdots & x_n \\
	\vdots & \vdots & & \vdots \\
	x_0^n & x_1^n & \cdots & x_n^n
	\end{pmatrix}.
$$
$\mathbf{V} \mathbf{x} = \mathbf{b}$ or $\mathbf{V}^T \mathbf{x} = \mathbf{b}$ can be solved in $O(n^2)$ flops.

## Cauchy-like matrix

Cauchy-like matrices:
$$
	\Omega \mathbf{A} - \mathbf{A} \Lambda = \mathbf{R} \mathbf{S}^T,
$$
where $\Omega = \text{diag}(\omega_1,\ldots,\omega_n)$ and $\Lambda = \text{diag}(\lambda_1,\ldots, \lambda_n)$. $O(n)$ flops for LU and QR.

## Structured-rank matrix

Structured-rank problems: semiseparable matrices (LU and QR takes $O(n)$ flops), quasiseparable matrices, ...

# LinearSolve.jl package

[LinearSolve.jl](https://github.com/SciML/LinearSolve.jl) is meta-package in Julia that defines a unified interface for solving linear equations and makes it easy switching linear solvers while maintaining the maximum efficiency.

TODO: examples.